In [1]:
from deepface import DeepFace
import pandas as pd
import numpy as np
from tqdm import tqdm

2023-04-12 17:58:53.210276: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-04-12 17:58:53.234696: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-12 17:58:53.577215: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
df = pd.read_csv('./fairface_label_val.csv')
df.head()

,file,age,gender,race,service_test
0,val/1.jpg,3-9,Male,East Asian,False
1,val/2.jpg,50-59,Female,East Asian,True
2,val/3.jpg,30-39,Male,White,True
3,val/4.jpg,20-29,Female,Latino_Hispanic,True
4,val/5.jpg,20-29,Male,Southeast Asian,False


In [4]:
pd.get_dummies(df.race).columns.values

array(['Black', 'East Asian', 'Indian', 'Latino_Hispanic',
       'Middle Eastern', 'Southeast Asian', 'White'], dtype=object)

In [3]:
age_dic = {'0-2':[0,2], '10-19':[10,19], '20-29':[20,29], '3-9':[3,9], '30-39':[30,39], '40-49':[40,49], '50-59':[50,59], '60-69':[60,69],
       'more than 70':[70,1000]}
gender_dic = {'Female':0, 'Male':1}
race_dic = {'Black':0, 'East Asian':1, 'Indian':2, 'Latino_Hispanic':3,
       'Middle Eastern':4, 'Southeast Asian':5, 'White':6}

In [4]:
all_results = np.zeros((len(gender_dic),len(race_dic)))
correct_results = np.zeros((len(gender_dic),len(race_dic)))
not_dect_results = np.zeros((len(gender_dic),len(race_dic)))

In [5]:
backends = [
  'opencv', 
  'ssd', 
  'dlib', 
  'mtcnn', 
  'retinaface', 
  'mediapipe'
]

for i in tqdm(range(len(df))):
    img = df.iloc[i].file
    true_age = df.iloc[i].age
    min_age, max_age = age_dic[true_age]
    gender = gender_dic[df.iloc[i].gender]
    race = race_dic[df.iloc[i].race]
    try:
        objs = DeepFace.analyze(img_path = df.iloc[i].file, 
            actions = ['age'], detector_backend = backends[4]
        )
        pred_age = objs[0]['age']
        if min_age < pred_age < max_age:
            correct_results[gender,race] += 1
        all_results[gender,race] += 1
    except:
        not_dect_results[gender,race] += 1


  0%|          | 0/10954 [00:00<?, ?it/s]2023-04-12 17:21:47.568117: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-04-12 17:21:47.585157: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-04-12 17:21:47.585273: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux

In [13]:
correct_results

array([[104., 116., 118., 126.,  65.,  90., 150.],
       [ 82., 120., 125., 136., 134., 130., 198.]])

In [7]:
all_results

array([[436., 568., 568., 622., 284., 506., 673.],
       [359., 482., 508., 551., 534., 508., 758.]])

In [14]:
correct_results/all_results

array([[0.23853211, 0.20422535, 0.20774648, 0.20257235, 0.22887324,
        0.17786561, 0.22288262],
       [0.22841226, 0.24896266, 0.24606299, 0.24682396, 0.25093633,
        0.25590551, 0.26121372]])

: 

In [9]:
np.sum(all_results)


7357.0

In [10]:
not_dect_results

array([[321., 205., 195., 208., 112., 174., 290.],
       [440., 295., 245., 242., 279., 227., 364.]])

In [11]:
acc = correct_results / all_results

In [12]:
acc

array([[0.23853211, 0.20422535, 0.20774648, 0.20257235, 0.22887324,
        0.17786561, 0.22288262],
       [0.22841226, 0.24896266, 0.24606299, 0.24682396, 0.25093633,
        0.25590551, 0.26121372]])

In [19]:
np.mean(acc, axis=0)

array([0.20599465, 0.29273256, 0.24859852, 0.20364772, 0.25973816,
       0.25275169, 0.26431496])

In [20]:
np.std(np.mean(acc, axis=0))

0.029640267755986088

In [27]:
np.log10((np.max(acc) / np.min(acc)))

0.2388043906446185

In [2]:
acc = np.array([[0.23853211, 0.20422535, 0.20774648, 0.20257235, 0.22887324,
        0.17786561, 0.22288262],
       [0.22841226, 0.24896266, 0.24606299, 0.24682396, 0.25093633,
        0.25590551, 0.26121372]])

In [7]:
np.mean(acc)

0.23007251357142858

In [10]:
np.log10(np.max(acc) / np.min(acc))

0.16690399796969654